In [6]:
import random
from scipy.stats import cauchy
import numpy as np

In [7]:
class Individual:
    def __init__(self, prob, dim, genes=None):
        self.genes = np.random.rand(dim) if genes is None else genes
        self.prob = prob
        self.dim = dim
        self.fitness = 0
        self.F = 0
        self.CR = 0
    def evaluate(self):
        self.fitness = self.prob(self.genes)
    def getFCR(m_f, m_cr):
        while(True):
            f = np.random.normal(loc = m_f, scale = 0.1)
            cr = cauchy.rvs(loc = m_cr, scale = 0.1, size = 1)[0]
            if f >= 0 and cr >=0:
                break
        self.F = min(f,1)
        self.CR = min(cr,1)
    def RandomMutate(self):
        k = np.array([True]* self.dim)
        k[random.randint(0, self.dim-1)] = False
        v = np.random.rand(self.dim)
        u = self.genes
        return np.clip(np.where(k, u, v), 0, 1)
    def BGAMutate(self):
        k = np.array([True]* self.dim)
        k[random.randint(0, self.dim-1)] = False
        v = np.random.rand(self.dim)
        random_values = np.random.rand(16)
        # alpha = np.sum(np.where(random_values < 0.5, 0, 2 ** -np.arange(16, dtype=float)))
        alpha = 0.1
        if random.random() < 0.5:
            random_vector = self.genes + alpha * np.random.rand(self.dim)
        else:
            random_vector = self.genes - alpha * np.random.rand(self.dim)
        return np.clip(np.where(k, self.genes, random_vector), 0, 1)



In [8]:
class Population:
    def __init__(self, pop_size, prob, dim):
        self.pop_size = pop_size
        self.prob = prob
        self.dim = dim
        self.pool = []
    def init(self):
        for i in range(self.pop_size):
            ind = Individual(genes = None, prob = self.prob, dim = self.dim)
            ind.evaluate()
            self.pool.append(ind)
    def best_fit(self):
        self.pool.sort(key = lambda x : x.fitness)
        return self.pool[0].fitness
    def restart(self):
        self.pool.sort(key = lambda x : x.fitness)
        self.pool = self.pool[:int(len(self.pool)/10)]
        
                

In [9]:
class TripleDE:
    def __init__(self, pop_size, pop_min, prob, dim, MAX_FES, H):
        self.pop_size = pop_size
        self.pop_min = pop_min
        self.prob = prob
        self.dim = dim
        self.MAX_FES = MAX_FES
        self.pop = Population(self.pop_size, self.prob, self.dim)
        self.pop.init()
        self.H = H
        self.m_cr = [0.9] * H  
        self.m_f = [0.3] * H
        self.p_selection = [1/H] * H
        self.num_selection = [0] * H 
        self.num_success = [0] * H
        self.archive = []
        self.success = False
        self.mem_pos = 0
        self.stay = 0
    def selectFCR(self):
        idx_list = list(range(self.H))
        random_idx = random.choices(idx_list, self.p_selection)[0]
        self.num_selection[random_idx] += 1
        return self.m_cr[random_idx], self.m_f[random_idx], random_idx

    def produce_worst(self, worst, medium, best, idx):
        donor = worst.genes + worst.F * (medium.genes - worst.genes) + worst.F * (best.genes - worst.genes)
        u = np.random.rand(self.dim) < worst.CR
        k = np.random.randint(self.dim)
        u[k] = True
        off_genes = np.where(u, donor, worst.genes)
        off_genes = np.clip(off_genes, 0, 1)
        off = Individual(genes = off_genes, prob = self.prob, dim = self.dim)
        off.evaluate()
        successF = []
        successCR = []
        improvement = []
        if off.fitness < worst.fitness:
            self.archive.append(worst)
            self.num_success[idx] += 1
            self.success = True
            successF.append(worst.F)
            successCR.append(worst.CR)
            improvement.append(worst.fitness - off.fitness)
        return off, successF, successCR, improvement

    def produce_medium(self, medium, best, idx):
        x1 = random.choice(self.pop.pool)
        _x2 = random.choice(self.pop.pool + self.archive)
        donor = medium.genes + medium.F * (best.genes - medium.genes) + medium.F * (x1.genes - _x2.genes)
        u = np.random.rand(self.dim) < medium.CR
        k = np.random.randint(self.dim)
        u[k] = True
        off_genes = np.where(u, donor, medium.genes)
        off_genes = np.clip(off_genes, 0, 1)
        off = Individual(genes = off_genes, prob = self.prob, dim = self.dim)
        off.evaluate()
        successF = []
        successCR = []
        improvement = []
        if off.fitness < medium.fitness:
            self.archive.append(medium)
            self.num_success[idx] += 1
            self.success = True
            successF.append(medium.F)
            successCR.append(medium.CR)
            improvement.append(medium.fitness - off.fitness)
        return off, successF, successCR, improvement

    def produce_best(self, best, idx):
        x1 = random.choice(self.pop.pool)
        _x2 = random.choice(self.pop.pool + self.archive)
        donor = best.genes + best.F * (x1.genes - _x2.genes)
        u = np.random.rand(self.dim) < best.CR
        k = np.random.randint(self.dim)
        u[k] = True
        off_genes = np.where(u, donor, best.genes)
        off_genes = np.clip(off_genes, 0, 1)
        off = Individual(genes = off_genes, prob = self.prob, dim = self.dim)
        off.evaluate()
        successF = []
        successCR = []
        improvement = []
        if off.fitness < best.fitness:
            self.archive.append(best)
            self.num_success[idx] += 1
            self.success = True
            successF.append(best.F)
            successCR.append(best.CR)
            improvement.append(best.fitness - off.fitness)
        return off, successF, successCR, improvement
    def generateFCR(self, m_f, m_cr):
        f_list = []
        cr_list = []
        for _ in range(3):
            while(True):
                f = np.random.normal(loc = m_f, scale = 0.1)
                cr = cauchy.rvs(loc = m_cr, scale = 0.1, size = 1)[0]
                if f >= 0 and cr >=0:
                    break
            f_list.append(min(f,1))
            cr_list.append(min(cr,1))
        return f_list, cr_list

    def updateF(self, success_F, success_improv):
        success_F = np.array(success_F)
        success_improv = np.array(success_improv)
        weight = success_improv / np.sum(success_improv)
        tu = np.sum(weight * success_F * success_F)
        mau = np.sum(weight * success_F)
        return tu/mau
    def updateCR(self, success_CR, success_improv):
        success_CR = np.array(success_CR)
        success_improv = np.array(success_improv)
        weight = success_improv / np.sum(success_improv)
        tu = np.sum(weight * success_CR * success_CR)
        mau = np.sum(weight * success_CR)
        return tu/mau
    def updateMemory(self, success_F, success_CR, success_improv):
        self.m_f[self.mem_pos] = self.updateF(success_F, success_improv)
        self.m_cr[self.mem_pos] = self.updateCR(success_CR, success_improv)
        for i in range(self.H):
            if self.num_selection[i] == 0:
                self.p_selection[i] = 0
            else:
                self.p_selection[i] = self.num_success[i]/self.num_selection[i]
        self.num_selection = [0] * self.H
        self.num_success = [0] * self.H 
        self.mem_pos = (self.mem_pos + 1) % self.H

    def run(self):
        FEs = self.pop_size
        while FEs <= self.MAX_FES:
            previous_fitness = self.pop.best_fit()
            self.success = False
            random.shuffle(self.pop.pool)
            N = len(self.pop.pool)
            success_F = []
            success_CR = []
            success_improv = []
            tri1, tri2, tri3 = self.pop.pool[:int(N/3)], self.pop.pool[int(N/3):int(2*N/3)], self.pop.pool[int(2*N/3):]
            triples = [[tri1[i], tri2[i], tri3[i]] for i in range(int(N/3))]
            new_pool = []
            for triple in triples:
                triple.sort(key = lambda x: x.fitness)
                best, medium, worst = triple
                m_f, m_cr, idx = self.selectFCR()
                f_list, cr_list = self.generateFCR(m_f, m_cr)
                f_list.sort()   
                cr_list.sort()
                best.F, best.CR = f_list[0], cr_list[0]
                medium.F, medium.CR = f_list[1], cr_list[1]
                worst.F, worst.CR = f_list[2], cr_list[2]
                best_off, suc_best_F, suc_best_CR, best_improv = self.produce_best(best, idx)
                medium_off, suc_medium_F, suc_medium_CR, medium_improv = self.produce_medium(medium, best, idx)
                worst_off, suc_worst_F, suc_worst_CR, worst_improv =self.produce_worst(worst, medium, best, idx)
                success_F.extend(suc_best_F + suc_medium_F + suc_worst_F)
                success_CR.extend(suc_best_CR + suc_medium_CR + suc_worst_CR)
                success_improv.extend(best_improv + medium_improv + worst_improv)
                FEs = FEs + 3 
                this_pool = triple + [best_off, medium_off, worst_off]
                this_pool.sort(key = lambda x: x.fitness)
                new_pool.extend(this_pool[:3])
            if not self.success:
                self.p_selection = [1/self.H] * self.H
            else:
                self.updateMemory(success_F, success_CR, success_improv)
            new_pop_size = 3 * int( (FEs/self.MAX_FES * (self.pop_min - self.pop_size) + self.pop_size) / 3)
            new_pool.sort(key = lambda x: x.fitness)
            self.pop.pool = new_pool[:new_pop_size]
            random.shuffle(self.archive)
            self.archive = self.archive[:int(1.4 * len(self.pop.pool))]
            current_fitness = self.pop.best_fit()
            if abs(previous_fitness - current_fitness) < 1e-4:
                self.stay += 1
            else:
                self.stay = 0
            print("FITNESS: ", current_fitness)
            
                


                
                

            


In [10]:
from gnbg_func.GNBG import GNBG

prob = GNBG(1).fitness_mfea

In [11]:
TripleDE(pop_size = 102, pop_min=21, prob = prob, dim = 30, MAX_FES = 30000, H=6).run()

FITNESS:  108216.49224421407
FITNESS:  80682.4547709853
FITNESS:  69701.12568218113
FITNESS:  69701.12568218113
FITNESS:  69701.12568218113
FITNESS:  68665.4646975347
FITNESS:  49029.04498035383
FITNESS:  49029.04498035383
FITNESS:  49029.04498035383
FITNESS:  49029.04498035383
FITNESS:  41848.45357517088
FITNESS:  41848.45357517088
FITNESS:  41848.45357517088
FITNESS:  37443.153852880605
FITNESS:  37443.153852880605
FITNESS:  37443.153852880605
FITNESS:  37443.153852880605
FITNESS:  37443.153852880605
FITNESS:  37443.153852880605
FITNESS:  35870.00394841064
FITNESS:  35870.00394841064
FITNESS:  26073.01836351456
FITNESS:  26073.01836351456
FITNESS:  26073.01836351456
FITNESS:  17604.300520835037
FITNESS:  17604.300520835037
FITNESS:  17604.300520835037
FITNESS:  17604.300520835037
FITNESS:  17604.300520835037
FITNESS:  17604.300520835037
FITNESS:  17604.300520835037
FITNESS:  17604.300520835037
FITNESS:  17604.300520835037
FITNESS:  17604.300520835037
FITNESS:  17604.300520835037
FITN